In [1]:
import argparse
import glob
import json
import logging
import os
import tarfile
import urllib.request

import librosa
import numpy as np
import soundfile as sf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import itertools  
df=pd.read_csv('combined_data.csv')
labels=df['covid_status'].unique()
len(labels), labels
l=len(labels)
num2labels=dict([(k,v) for k,v in zip(range(l),labels)])
print(num2labels)
df['wav_path']=''
root='/workspace/Coswara-Data/data/'
folders=os.listdir(root)
folders=[f for f in folders if f.startswith('2020')]
ids=list(df['id'].values)
covid_labels=list(df['covid_status'].values)
wav_files_ls=[]
for folder in folders:
    cur_dir=root+folder+'/'
    #print(cur_dir)
    sub_folders=os.listdir(cur_dir)
    subfolders=[subf for subf in sub_folders if subf in ids]
    #print(subfolders)
    templist=[ cur_dir+t+'/'+'cough-heavy.wav' for t in subfolders if os.path.exists(cur_dir+t+'/'+'cough-heavy.wav')]
    wav_files_ls.append(templist)
wav_files_ls = list(itertools.chain(*wav_files_ls))
print("original number of records ",len(df))
assert len(ids)==len(df)
len(ids)
## put the wav file path into the dataframe         

for wav_fpath in wav_files_ls:
    wav_fname=wav_fpath.split('/')[-2]
    if wav_fname in ids:
        k=ids.index(wav_fname)
        df.iloc[k,-1]=wav_fpath
#verify all wav files exist
df2=df[~df['wav_path'].isnull()]
print(len(df2))

valid_cols=['id', 'a', 'covid_status','wav_path']
df2=df2[valid_cols]
labels2num=dict([(v,k) for (k,v) in num2labels.items()] )

df2['covid_status'].unique()
len(df2)

{0: 'healthy', 1: 'resp_illness_not_identified', 2: 'no_resp_illness_exposed', 3: 'recovered_full', 4: 'positive_mild', 5: 'positive_asymp', 6: 'positive_moderate'}
original number of records  1503
1503


1503

In [2]:
del df

In [3]:
labels2num

{'healthy': 0,
 'resp_illness_not_identified': 1,
 'no_resp_illness_exposed': 2,
 'recovered_full': 3,
 'positive_mild': 4,
 'positive_asymp': 5,
 'positive_moderate': 6}

In [4]:
labels=list(df2['covid_status'].values)
df2['covid_status'].replace(labels2num, inplace=True)
df2['labels']=labels

In [5]:
df2.head()

,id,a,covid_status,wav_path,labels
0,DRBAZX64nuVtqBQf13gH7r36Mh52,26,0,/workspace/Coswara-Data/data/20200430/DRBAZX64...,healthy
1,Jw7YMfwGqMX22UbHh1TTgYMTYWs1,16,0,/workspace/Coswara-Data/data/20200430/Jw7YMfwG...,healthy
2,xa2v8z3Yzgb9dFrq2gEZz6oS7fh1,26,1,/workspace/Coswara-Data/data/20200430/xa2v8z3Y...,resp_illness_not_identified
3,xwHQrG0KwjTLJvBYVtVXfHp4JAd2,32,1,/workspace/Coswara-Data/data/20200430/xwHQrG0K...,resp_illness_not_identified
4,xKW2EpGmJcfigLecUTLVXEWQHPg2,22,0,/workspace/Coswara-Data/data/20200430/xKW2EpGm...,healthy


In [6]:
labels2binary = {0:'0', 1: '0', 2: '0', 3:'1', 4:'1',5:'1',6:'1'}
binary_label=df2.replace({"covid_status": labels2binary},)


In [7]:
assert len(binary_label)==len(df2)

In [8]:
df2['binary_label']=binary_label.covid_status
df2.head()

,id,a,covid_status,wav_path,labels,binary_label
0,DRBAZX64nuVtqBQf13gH7r36Mh52,26,0,/workspace/Coswara-Data/data/20200430/DRBAZX64...,healthy,0
1,Jw7YMfwGqMX22UbHh1TTgYMTYWs1,16,0,/workspace/Coswara-Data/data/20200430/Jw7YMfwG...,healthy,0
2,xa2v8z3Yzgb9dFrq2gEZz6oS7fh1,26,1,/workspace/Coswara-Data/data/20200430/xa2v8z3Y...,resp_illness_not_identified,0
3,xwHQrG0KwjTLJvBYVtVXfHp4JAd2,32,1,/workspace/Coswara-Data/data/20200430/xwHQrG0K...,resp_illness_not_identified,0
4,xKW2EpGmJcfigLecUTLVXEWQHPg2,22,0,/workspace/Coswara-Data/data/20200430/xKW2EpGm...,healthy,0


In [9]:
del binary_label

In [10]:
def check_label_correctly(txt,b,a,labels2num,labels2binary, index):
    if labels2num[txt]==b and labels2binary[b]==a:
        pass
    else:
        print("index, label :{} , num_label:{} shirnk to binary:{}".format(str(index),txt,str(b),str(a)))
idx=0
for txt,before,after in zip(df2['labels'],df2['covid_status'],df2['binary_label']):
    check_label_correctly(txt,before,after,labels2num,labels2binary, idx)
    idx+=1


In [11]:
keep_column=['wav_path', 'labels', 'binary_label']
X=df2[keep_column[:-1]].values
y=df2['binary_label'].values
type(X),type(y)

(numpy.ndarray, numpy.ndarray)

In [12]:
for file in X:
    if file[1]=='healthy':
        print(file)
        break

['/workspace/Coswara-Data/data/20200430/DRBAZX64nuVtqBQf13gH7r36Mh52/cough-heavy.wav'
 'healthy']


In [23]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, random_state=1,stratify=y)
X_train.shape, X_test.shape, y_train.shape,y_test.shape

((1202, 2), (301, 2), (1202,), (301,))

In [24]:
txtlabels2binary={'healthy': 'healthy',
 'resp_illness_not_identified': 'healthy',
 'no_resp_illness_exposed': 'healthy',
 'recovered_full': 'covid_positive',
 'positive_mild': 'covid_positive',
 'positive_asymp': 'covid_positive',
 'positive_moderate': 'covid_positive'}
txtlabels2binary

{'healthy': 'healthy',
 'resp_illness_not_identified': 'healthy',
 'no_resp_illness_exposed': 'healthy',
 'recovered_full': 'covid_positive',
 'positive_mild': 'covid_positive',
 'positive_asymp': 'covid_positive',
 'positive_moderate': 'covid_positive'}

In [ ]:
#sox disturbence.wav -r 16000 -c 1 -b 16 disturbence_16000_mono_16bit.wav

In [31]:
import argparse
import glob
import json
import logging
import os
import tarfile
import librosa
import numpy as np
import re
in_wav='/workspace/Coswara-Data/data/20200430/DRBAZX64nuVtqBQf13gH7r36Mh52/cough-heavy.wav'
def change_audio_sampling_rate(in_wav, do_train=True):
    in_wav_pathsub='_'.join(in_wav.split('/')[4:-1])
    if do_train:
        out_path='/workspace/NeMo/dataset/train/'
    else:
        out_path='/workspace/NeMo/dataset/test/'
    in_wav_name=in_wav.split('/')[-1].split('.')[0]
    out_wav_path=out_path+in_wav_pathsub+in_wav_name+'_16000_mono_16bit.wav'
    os.system("sox {} -r 16000 -c 1 -b 16 {}".format(in_wav,out_wav_path))
    return out_wav_path


In [26]:
X[0]

array(['/workspace/Coswara-Data/data/20200430/DRBAZX64nuVtqBQf13gH7r36Mh52/cough-heavy.wav',
       'healthy'], dtype=object)

In [33]:

def make_manifest(manifest_name,X,Y, y_num=False, do_train=True):
    out_dir='/workspace/NeMo/dataset'    
    sr = 16000
    start=0.0
    end=None
    duration_stride=1.0
    duration_max = 1e9
    duration_limit=100.0
    filter_long=False
    seg_num = 0
    skip_num = 0
    if not os.path.exists(out_dir):
        logging.info(f'Outdir {out_dir} does not exist. Creat directory.')
        os.mkdir(out_dir)
    output_path = os.path.join(out_dir, manifest_name + '.json')
    with open(output_path, 'w') as fout:
        for x,y in zip(X,Y):
            file=change_audio_sampling_rate(x[0], do_train)
            if y_num:
                label=y
            else:
                label=txtlabels2binary[x[1]]
            try:
                x, _sr = librosa.load(file, sr=sr)
                duration = librosa.get_duration(x, sr=sr)
                #print(x.shape,_sr,duration)

            except Exception:
                continue       

            
            metadata = {
                        'audio_filepath': file,
                        'duration': duration,
                        'label': label,
                        'text': '_',  # for compatibility with ASRAudioText
                        }
            data=json.dump(metadata,fout)            
            fout.write('\n')
            fout.flush()
    print("finished creating manifest file :", manifest_name+'.json')
#manifest_name='train_numLabels'
#manifest_name='train_txtLabels'
do_train=False
#manifest_name='test_numLabels'
manifest_name='test_txtLabels'
make_manifest(manifest_name,X_test,y_test,False, do_train)

finished creating manifest file : test_txtLabels.json
